## Reinforcement Learning

Рекомендованная литература:

* Barto Sutton Reinforcement Learning: An Introduction (классика теории)
* Lapan Deep Reinforcement Learning Hands-On Second Edition (Книга с большим количеством примеров по RL)
* Sudharsan Ravichandiran Hands-On Reinforcement Learning with Python (Тоже много примеров)

Прочая дополнительная литература:

* Alex Zai and Brandon Brown Deep Reinforcement Learning in Action


Полезные ссылки:
* https://www.alexirpan.com/2018/02/14/rl-hard.html Deep Reinforcement Learning Doesn't Work Yet
* https://openai.com/blog/faulty-reward-functions/  Faulty Reward Functions in the Wild

## Intro



### Обучение с учителем
Если у нас есть набор примеров с правильными ответами, то мы используем эту выборку для обучения нашей модели, а после обучения, применяем её к неразмеченным данным. Именно этот подход мы использовали, когда обучали классификатор для MNIST, подавая на вход сети картинки с изображениями рукописных цифр и считая градиент для подстройки весов на основе разницы между известным лэйблом цифры и выходом нейросети.
### Обучение без учителя
В некоторых случаях у нас нет размеченных данных, на которых мы могли бы заранее обучить модель. Но, при решении некоторых задач, можно обойтись без размеченной выборки. Примером такой задачи является задача кластеризации.
### Обучение с подкреплением
В некоторых случаях существующие методы обучения без учителя нам не подходят. В то же время у нас нет возможности создать качественную обучающую выборку. При этом мы можем постфактум оценить действия нашей модели и использовать эту оценку подстроить модель так, чтобы она чаще совершала желательные действия и реже - нежелательные. В литературе такую оценку называют вознаграждением (reward), а обучение строится таким образом, чтобы это модель стремилась максимизировать получаемое вознаграждение.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-7.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## Терминология: агент, функция награды, состояние среды

Агент и среда - ключевые понятия в обучении с подкрелением.

**Агент** - программа, принимающая решение о дальнейших действиях на основе информации о состоянии среды.

**Среда** - это мир, в котором агент должен "выживать", т.е. всё, с чем агент может прямо или косвенно взаимодействовать. Среда обладает состоянием (State), агент может влиять на среду, совершая какие-то действия (Actions), переводя среду при этом из одного состояния в другое и получая какое-то вознаграждение. Среда описывается пространством возможных состояний. Конкретное состояние - вектор в этом пространстве.

В зависимости от конкретной задачи, агент может наблюдать либо полное состояние среды, либо только некоторую его часть. Во втором случае, агенту может потребоваться какое-то внутреннее представление полного состояния, которое будет обновляться по мере получения новых данных.

**Фукнция награды** - вводимая программистом формула вычисления ценности действия на основе финального результата, ожидания этого результата, промежуточных результатов и любых других параметров, которые будут подсказывать путь к наилучшей последовательности действий агента. Это некоторый аналог функции потерь, без которой непонятно чему учиться. Например, в шахматах истинная награда это победа, но взятая фигура соперника тоже ценна и должна увеличивать награду, если мы хотим подсказать агенту, что брать чужие фигуры полезно. Может ли после этого агент получать мат, позарившись на незащищенную фигуру? Да, ровно как и неопытный шахматный игрок. Попытка передать через дополнительные неосновные награды подсказки к получению основной награды называется **reward shaping**.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-8.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-9.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

### MDP

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-24_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-4.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-3.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-25_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-12.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-13.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-15.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-26.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-28.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-29_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-30.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-31.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-33_.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-35__.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-32_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-34_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-36.png" alt="Drawing" style="width: 500px;"/>

### Давайте теперь искать наилучшую политику или наилучший способ поведения

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-37.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-38_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-39_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-40.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-41.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-42_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-43.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-45_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-44_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-46_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-47_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-48.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-49.png" alt="Drawing" style="width: 500px;"/>

## Bellman

### Policy evaluation

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-56.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-67.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-9.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-10.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-11.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

### Policy improvement

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-60.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-13.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-72.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-75.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## Библиотека OpenAI Gym

В OpenAI создали готовую библиотеку для моделировавния, визуализирования и обучения, что не только удобно для тестов. Одно из хороших преимуществ это качественный абстрактный интерфейс взаимодействия среды и агента, который позволяет делать совместимые с Gym среды и впоследствии интегрироваться в экосистему, легко подменять типы агентов, тестировать, сравнивать.
Среди готовых сред для взаимодействия с ними есть
- игры Atari - полноценные компьютерные игры, типа Арканоида, где вместо человека в компьютер играет агент.
- классические настольные пошаговые игры: шахматы, go.
- физические симуляции, где нужно управлять физическим объектом: маятник, который должен перевернуться и балансировать за счёт раскачивания; тележка, которая должна проехать через холм за счёт раскачивания.
- и т.д.### State-value function

### Пример окружения Gym
В этом примере загружается готовая среда: машинка должна заехать на горку. Функция награды встроенная. А агент - случайное воздействие. Оно ничему не учится, лишь хаотически выдаёт действия, но это показывает где взять все необходимые данные для обучения.

In [1]:
import gym
env = gym.make("MountainCar-v0")
MAX_NUM_EPISODES = 5

for episode in range(MAX_NUM_EPISODES):
    done = False
    obs = env.reset()
    total_reward = 0.0 # To keep track of the total reward obtained in each episode
    step = 0
    while not done:
        env.render()
        
        
        
        action = env.action_space.sample()  # Sample random action.
                                            # This will be replaced
                                            # by our agent's action
                                            # when we # start
                                            # developing the agent algorithms
        
        
        next_state, reward, done, info = \
        env.step(action)  # Send the action to the
                          # environment and receive       
                          # the next_state, reward and
                          # whether done or not
        total_reward += reward
        step += 1
        obs = next_state
    print("\n Episode #{} ended in {} steps. total_reward={}".format(episode, step+1,
total_reward))
env.close()


 Episode #0 ended in 201 steps. total_reward=-200.0

 Episode #1 ended in 201 steps. total_reward=-200.0

 Episode #2 ended in 201 steps. total_reward=-200.0

 Episode #3 ended in 201 steps. total_reward=-200.0

 Episode #4 ended in 201 steps. total_reward=-200.0


## Temporal-Difference Learning

### Q-learning

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-14.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-16.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-19.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-20.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-24.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-25.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-27.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-28.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-31.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-32.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-33.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-34.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-36.png" alt="Drawing" style="width: 500px;"/> 


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-37.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-39.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-40.png" alt="Drawing" style="width: 430px;"/> 


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-44.png" alt="Drawing" /> 

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-48.png" alt="Drawing" /> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-50.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-52.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-53.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-54.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-59.png" alt="Drawing" style="width: 450px;"/> 

## yds-4

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-6.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-8.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-9.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-10.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-11.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-12.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/coursera-4-24.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/coursera-4-25.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-14.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-15.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-16.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## Experience replay

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-18.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-62.png" alt="Drawing" /> </td>
</tr></table>

В большинстве окружений информация, получаемая агентом распределена не независимо. Т.е. последовательные наблюдения агента сильно коррелированы между собой (что понятно из интуитивных соображений, т.к. большинство окружений, в которых применяется RL, предполагают, что все изменения в них последовательны). Корреляция примеров ухудшает сходимость стохастического градиентного спуска. Таким образом нам нужен способ, который позволяет улучшить распределение примеров для обучения (устранить или снизить корреляцию между ними). Обычно используется метод **проигрывания опыта (experience replay)**. Суть этого метода в том, что мы сохраняем некоторое количество примеров (состояние, действия, вознаграждение) в специальном буфере и для обучения выбираем случайные мини-батчи из этого буфера.

Так же **experience replay** позволяет агенту эффективнее использовать свой прошлый опыт.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-23.png" alt="Drawing" style="width: 500px;"/> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-24.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-25.png" alt="Drawing" /> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-26.png" alt="Drawing" /> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-28.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-29.png" alt="Drawing" /> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-30.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-31.png" alt="Drawing" /> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-32.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-33.png" alt="Drawing" /> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-34.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-35.png" alt="Drawing" /> </td>
</tr></table>

## Double DQN


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-36.png" alt="Drawing" style="width: 500px;"/> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-37.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-38.png" alt="Drawing" /> </td>
</tr></table>

Одной из проблем Q-Networks является неустойчивость. Часто разность ожидаемых вознаграждений для различных действий близка и поскольку выбор действия производится с помощью argmax, то выброс в данных может привести к тому, что выбираемое действие изменится. Для того, чтобы повысить стабильность используется техника **Target Q-Network**. Суть в том, что мы замораживаем веса нашей сети на фиксированное число шагов и затем используем её для вычисления функции ошибки и обучения второй сети. Периодически копируем из веса рабочей сети в Target Q-Network.
В следующем примере мы будем делать это каждый эпизод.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-39.png" alt="Drawing" style="width: 500px;"/> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-48.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-49.png" alt="Drawing" /> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-50.png" alt="Drawing" style="width: 500px;"/> 

# Другие улучшения DQN

## Prioritized Experience Replay

Минибатчи из памяти выбираются не с равномерным распределением, а добавляем туда больше примеров, в которых предсказанные значения Q сильнее всего отличаются от корректных. Т.е. примеры с максимальным **TD error** получают максимальный приоритет.

## Dueling networks

Основная идея в том, что мы разделяем нашу сеть на две головы, одна из которых предсказывает абсолютное значение состояния \\( V(S) \\), а вторая - относительное преимущество одний действий над другими \\( A(s, a) = Q(s, a) - V(s) \\). Это преимущество называется advantage. Далее из этих двух значений мы собираем нашу Q-функцию, как \\( Q(s,a) = V(s) + A(a) \\)


## Multi-step learning/n-step learning

Основная идея в том, чтобы считать функцию ценности не по двум соседним примерам, а сразу по n. Это позволяет сети лучше запоминать длинные последовательности действий.

## Distributional RL

Детерминированное значение Q заменяется случайным распределением Z с некоторыми параметрами, которые определяются в ходе обучения.

# Rainbow

Rainbow - набор перечисленных выше твиков. 

# R2D2

Использование RNN сетей для experience replay

## Резюме по типам алгоритмов RL

### Model-free / Model-based
Model-free не строит модель окружения или функции награды. Это проще. Model-based алгоритм пытается предсказывать, каким будет следующее состояние окружения или вознаграждение. Это позволяет мыслить на несколько шагов вперёд. Например, совершенно не больно прыгать с крыши. Больно потом, когда разбиваешься о землю. Модель среды и награды позволяет принимать решения не прыгать с крыши, хотя и model-free подход позволяет это понять, хоть и более сложно и грубо.
### Value-based / policy-based
Policy-based  методы оптимизируют напрямую функцию принятия решения агента. Стратегия (policy) обычно представлена распределением вероятности доступных действий. Value-based метод оптимизирует оценку вознаграждения для всех действий и выбирает выбирает то действие, по которому прогнозируется большее значение. Методы, основанные на Policy Gradients лучше работают при большой размерности пространства действий, а Value-based методы, такие, как Deep Q-Learning требуют меньшего количества повторений для сходимости при малой размерности.
### On-Policy / Off-Policy
Off-policy подход позволяет учиться на исторических данных или на записанных заранее действиях человека. On-policy - только на собственных действиях агента. Это довольно важное деление, так как обучение на собственных действиях просто недоступно для многих задач. Плохо ли автомобилю с автопилотом въехать в стену? Надо попробовать, так будет работать on-policy метод. А лучше взять понимание, что это плохо из готовых данных, синтетических, например и усвоить этот опыт. Даже если мы используем симуляцию среды, а не реальную среду, то количество эпизодов симуляции обычно порядка сотен тысяч и их симуляция съедает много времени. Каждый раз проходить переобучение с нуля очень неудобно, а возможность импортировать накопленный опыт симуляции среды сильно экономит ресурсы. Однако off-policy методы можно применить не всегда.
### Deterministic Policy / Stochastic Policy
В зависимости от среды, наша стратегия может быть либо детерминированной - выбираем сразу определённое действие с помощью argmax, либо стохастической, когда мы окончательное решение принимается с помощью генератора случайных чисел на основе распределения вероятности, выданного сетью.

### SARSA

## DQN

## C51
## Monte-Carlo Tree Search
Silver L8